In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
from matplotlib import pyplot as plt
import seaborn as sns
import random
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv


In [2]:
#!pip install iterative-stratification

In [3]:
data_path = "/kaggle/input/lish-moa/"
train_features = pd.read_csv(data_path + "train_features.csv")
train_targets = pd.read_csv(data_path + "train_targets_scored.csv")
test_features = pd.read_csv(data_path + "test_features.csv")
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

In [4]:
feature_names = train_features.columns
target_names = sample_submission.columns

In [5]:
submission = sample_submission.copy()
submission.set_index('sig_id', inplace=True)
submission.loc[:,:] = 1e-15
submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,...,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15
id_001897cda,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,...,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15
id_002429b5b,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,...,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15
id_00276f245,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,...,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15
id_0027f1083,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,...,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15,1.000000e-15


In [6]:
def preprocessing(features, targets=None):
    #indices to retain for training
    ret_indices = features['cp_type'] != 'ctl_vehicle'
    
    #process features
    X = features.copy()
    X = X.loc[ret_indices, :]
    id_ = X['sig_id']
    X.drop(columns=['sig_id', 'cp_type'], inplace=True)
    X = pd.get_dummies(X, columns=['cp_time', 'cp_dose'])
    
    if targets is not None:
        Y = targets.copy().loc[ret_indices, :]
        Y.drop(columns=['sig_id'], inplace=True)
        
        return id_, X, Y
    else:
        
        return id_, X

In [7]:
id_train, X_train, Y_train = preprocessing(train_features, train_targets)
id_test, X_test = preprocessing(test_features)

In [8]:
X_train.head()

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-95,c-96,c-97,c-98,c-99,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,...,0.6584,-0.3981,0.2139,0.3801,0.4176,1,0,0,1,0
1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,...,0.4899,0.1522,0.1241,0.6077,0.7371,0,0,1,1,0
2,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,...,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,0,1,0,1,0
3,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,...,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,1,0,1,0
4,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,...,-0.3031,0.1094,0.2885,-0.3786,0.7125,0,0,1,0,1


In [9]:
X_test.head()

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c-95,c-96,c-97,c-98,c-99,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
0,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,-0.2140,0.2221,-0.3260,1.9390,...,-0.1193,0.0210,-0.0502,0.1510,-0.7750,1,0,0,1,0
1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,-1.8820,0.4022,-0.3528,0.1271,...,-0.5382,0.0359,-0.4764,-1.3810,-0.7300,0,0,1,1,0
3,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,0.5998,-0.1799,0.9365,0.2942,...,-0.9005,0.8131,-0.1305,0.5645,-0.5809,1,0,0,0,1
4,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,0.5128,0.6365,0.2611,-1.1120,...,1.0900,-0.2962,-0.5313,0.9931,1.8380,0,1,0,1,0
6,0.3658,0.5536,-0.6898,-1.6270,0.5239,-0.3832,-0.4653,1.0070,0.3726,0.0811,...,-0.6570,-0.2593,-0.2174,0.0044,-1.4650,0,1,0,0,1


In [10]:
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
#from tensorflow.keras.layers import LeakyReLu
from tensorflow_addons.layers import WeightNormalization

prod_cols = lambda df, cols: pd.DataFrame(df[cols].prod(axis=1))
var_cols = lambda df, cols: pd.DataFrame(df[cols].var(axis=1))
mean_cols = lambda df, cols: pd.DataFrame(df[cols].mean(axis=1))
quantiles_cols = lambda df, cols, q: pd.DataFrame(df[cols].quantile(q, axis=1)).transpose()
skew_cols = lambda df, cols: pd.DataFrame(df[cols].skew(axis=1))
kurt_cols = lambda df, cols: pd.DataFrame(df[cols].kurtosis(axis=1))


def generate_X_pca(X_tr, X_val, X_test, num_comp_g=None, num_comp_c=None):
    N_tr = X_tr.shape[0]
    N_val = X_val.shape[0]
    N_test = X_test.shape[0]
    
    g_cols = list(X_tr.columns[list(map(lambda x: x[0:2]=='g-', X_tr.columns))])
    c_cols = list(X_tr.columns[list(map(lambda x: x[0:2]=='c-', X_tr.columns))])
    other_cols = list(set(X_tr.columns) - set(g_cols  + c_cols))
    
    #Create columns for values of PC transformation
    pca_g = PCA()
    X_tr_pca_g = pca_g.fit_transform(X_tr[g_cols])[:,0:num_comp_g]
    X_val_pca_g = pca_g.transform(X_val[g_cols])[:,0:num_comp_g]
    X_test_pca_g = pca_g.transform(X_test[g_cols])[:,0:num_comp_g]
    
    pca_c = PCA()
    X_tr_pca_c = pca_c.fit_transform(X_tr[c_cols])[:,0:num_comp_c]
    X_val_pca_c = pca_c.transform(X_val[c_cols])[:,0:num_comp_c]
    X_test_pca_c = pca_c.transform(X_test[c_cols])[:,0:num_comp_c]
    
    #Columns for one-hot encoded variables
    X_tr_others = X_tr[other_cols]
    X_val_others = X_val[other_cols]
    X_test_others = X_test[other_cols]
    
    
    #Columns for variance of g and of c variables
    X_tr_g_var = var_cols(X_tr, g_cols)
    X_val_g_var = var_cols(X_val, g_cols)
    X_test_g_var = var_cols(X_test, g_cols)
    
    X_tr_c_var = var_cols(X_tr, c_cols)
    X_val_c_var = var_cols(X_val, c_cols)
    X_test_c_var = var_cols(X_test, c_cols)
    
    #Columns for mean g and of c variables
    X_tr_g_mean = mean_cols(X_tr, g_cols)
    X_val_g_mean = mean_cols(X_val, g_cols)
    X_test_g_mean = mean_cols(X_test, g_cols)
    
    X_tr_c_mean = mean_cols(X_tr, c_cols)
    X_val_c_mean = mean_cols(X_val, c_cols)
    X_test_c_mean = mean_cols(X_test, c_cols)
    
    #Columns for the difference between the mean of g variables and that of c variables
    X_tr_mean_diff = X_tr_g_mean - X_tr_c_mean
    X_val_mean_diff = X_val_g_mean - X_val_c_mean
    X_test_mean_diff = X_test_g_mean - X_test_c_mean
    
    #Columns for quantiles of g and of c variables
    q = np.linspace(0, 1, num=22)
    
    X_tr_g_quantiles = quantiles_cols(X_tr, g_cols, q)
    X_val_g_quantiles = quantiles_cols(X_val, g_cols, q)
    X_test_g_quantiles = quantiles_cols(X_test, g_cols, q)
    
    X_tr_c_quantiles = quantiles_cols(X_tr, c_cols, q)
    X_val_c_quantiles = quantiles_cols(X_val, c_cols, q)
    X_test_c_quantiles = quantiles_cols(X_test, c_cols, q)
    
    #Columns for skewness of g and of c variables
    X_tr_g_skew = skew_cols(X_tr, g_cols)
    X_val_g_skew = skew_cols(X_val, g_cols)
    X_test_g_skew = skew_cols(X_test, g_cols)
    
    X_tr_c_skew = skew_cols(X_tr, c_cols)
    X_val_c_skew = skew_cols(X_val, c_cols)
    X_test_c_skew = skew_cols(X_test, c_cols)
    
    #Columns for kurtosis of g and of c variables
    X_tr_g_kurt = kurt_cols(X_tr, g_cols)
    X_val_g_kurt = kurt_cols(X_val, g_cols)
    X_test_g_kurt = kurt_cols(X_test, g_cols)
    
    X_tr_c_kurt = kurt_cols(X_tr, c_cols)
    X_val_c_kurt = kurt_cols(X_val, c_cols)
    X_test_c_kurt = kurt_cols(X_test, c_cols)
    
    
    #Columns for mean * cp time
    #24
    X_tr_g_mean_24 = np.exp(X_tr_g_mean) * np.array(X_tr['cp_time_24']).reshape((N_tr, 1))
    X_val_g_mean_24 = np.exp(X_val_g_mean) * np.array(X_val['cp_time_24']).reshape((N_val, 1))
    X_test_g_mean_24 = np.exp(X_test_g_mean) * np.array(X_test['cp_time_24']).reshape((N_test, 1))
    
    X_tr_c_mean_24 = np.exp(X_tr_c_mean) * np.array(X_tr['cp_time_24']).reshape((N_tr, 1))
    X_val_c_mean_24 = np.exp(X_val_c_mean) * np.array(X_val['cp_time_24']).reshape((N_val, 1))
    X_test_c_mean_24 = np.exp(X_test_c_mean) * np.array(X_test['cp_time_24']).reshape((N_test, 1))
    
    #48
    X_tr_g_mean_48 = np.exp(X_tr_g_mean) * np.array(X_tr['cp_time_48']).reshape((N_tr, 1))*2
    X_val_g_mean_48 = np.exp(X_val_g_mean) * np.array(X_val['cp_time_48']).reshape((N_val, 1))*2
    X_test_g_mean_48 = np.exp(X_test_g_mean) * np.array(X_test['cp_time_48']).reshape((N_test, 1))*2
    
    X_tr_c_mean_48 = np.exp(X_tr_c_mean) * np.array(X_tr['cp_time_48']).reshape((N_tr, 1))*2
    X_val_c_mean_48 = np.exp(X_val_c_mean) * np.array(X_val['cp_time_48']).reshape((N_val, 1))*2
    X_test_c_mean_48 = np.exp(X_test_c_mean) * np.array(X_test['cp_time_48']).reshape((N_test, 1))*2
    
    #72
    X_tr_g_mean_72 = np.exp(X_tr_g_mean) * np.array(X_tr['cp_time_72']).reshape((N_tr, 1))*3
    X_val_g_mean_72 = np.exp(X_val_g_mean) * np.array(X_val['cp_time_72']).reshape((N_val, 1))*3
    X_test_g_mean_72 = np.exp(X_test_g_mean) * np.array(X_test['cp_time_72']).reshape((N_test, 1))*3
    
    X_tr_c_mean_72 = np.exp(X_tr_c_mean) * np.array(X_tr['cp_time_72']).reshape((N_tr, 1))*3
    X_val_c_mean_72 = np.exp(X_val_c_mean) * np.array(X_val['cp_time_72']).reshape((N_val, 1))*3
    X_test_c_mean_72 = np.exp(X_test_c_mean) * np.array(X_test['cp_time_72']).reshape((N_test, 1))*3
    
    
    
    
    #Merge
    new_X_tr = np.concatenate([X_tr_others, X_tr_pca_g, X_tr_pca_c, #X_tr[g_cols+c_cols], 
                              X_tr_g_var, X_tr_c_var, X_tr_g_mean, X_tr_c_mean, X_tr_mean_diff,
                              X_tr_g_quantiles, X_tr_c_quantiles, X_tr_g_skew, X_tr_c_skew,
                              X_tr_g_kurt, X_tr_c_kurt,
                              X_tr_g_mean_24, X_tr_c_mean_24,
                              X_tr_g_mean_48, X_tr_c_mean_48,
                              X_tr_g_mean_72, X_tr_c_mean_72], axis=1)
    
    new_X_val = np.concatenate([X_val_others, X_val_pca_g, X_val_pca_c, #X_val[g_cols+c_cols],
                               X_val_g_var, X_val_c_var, X_val_g_mean, X_val_c_mean, X_val_mean_diff,
                               X_val_g_quantiles, X_val_c_quantiles, X_val_g_skew, X_val_c_skew,
                               X_val_g_kurt, X_val_c_kurt,
                               X_val_g_mean_24, X_val_c_mean_24,
                               X_val_g_mean_48, X_val_c_mean_48,
                               X_val_g_mean_72, X_val_c_mean_72], axis=1)
    
    new_X_test = np.concatenate([X_test_others, X_test_pca_g, X_test_pca_c, #X_test[g_cols+c_cols],
                                X_test_g_var, X_test_c_var, X_test_g_mean, X_test_c_mean, X_test_mean_diff,
                                X_test_g_quantiles, X_test_c_quantiles, X_test_g_skew, X_test_c_skew,
                                X_test_g_kurt, X_test_c_kurt,
                                X_test_g_mean_24, X_test_c_mean_24,
                                X_test_g_mean_48, X_test_c_mean_48,
                                X_test_g_mean_72, X_test_c_mean_72], axis=1)
    
    print("Number of features: {}".format(new_X_tr.shape[1]))
        
    return new_X_tr, new_X_val, new_X_test
        
    

In [11]:
from sklearn.model_selection import StratifiedKFold
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import LearningRateScheduler


def scheduler(epoch, lr):
    if epoch < 5 or lr <= 0.00001:
        return lr
    elif epoch < 100:
        return lr/1.05
    else:
        return lr

def run_model_kfold(X_train, Y_train, X_test, test_id, num_folds=5, num_comp_g=200, num_comp_c=20):
    #mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle=True)
    #for train_indices, val_indices in mskf.split(X_train, Y_train):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=2020)
    val_loss_hist = []
    
    for train_indices, val_indices in kf.split(X_train, Y_train):
 
        X_tr, X_val, X_test_pca = generate_X_pca(X_train.iloc[train_indices,:], 
                                             X_train.iloc[val_indices,:], 
                                             X_test, 
                                             num_comp_g=num_comp_g, 
                                             num_comp_c=num_comp_c)
    
        Y_tr, Y_val = Y_train.iloc[train_indices,:], Y_train.iloc[val_indices,:]
        
        
        
        model = create_model(X_tr.shape[1], Y_train.shape[1])
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
        lr_decay = LearningRateScheduler(scheduler)
        
        hist = model.fit(X_tr, Y_tr, 
                  validation_data = (X_val, Y_val),
                  batch_size=512, epochs=200, callbacks=[es, lr_decay])
        
        val_loss_hist.append(hist.history['val_loss'][-1])
        
        
        submission.loc[test_id,:] += model.predict(X_test_pca)/num_folds
    
    print("Validation Losses: {}".format(val_loss_hist))
    print("Mean Validation Loss: {}".format(np.mean(val_loss_hist)))
        

In [12]:
from tensorflow.keras.layers import BatchNormalization
def create_model(input_shape, num_outputs):
    model = keras.Sequential([
        
        keras.Input(shape=input_shape),
        BatchNormalization(),
        
        Dense(64, activation='relu'), 
        BatchNormalization(),
        
        
        Dense(num_outputs, activation='sigmoid') 
    ])
    
    opt = keras.optimizers.Adam(learning_rate=0.005)
    
    model.compile(loss='binary_crossentropy', optimizer=opt)
    
    return model

In [13]:
num_comp_g = 60
num_comp_c = 10
run_model_kfold(X_train, Y_train, X_test, id_test, num_folds=20, num_comp_g=num_comp_g, num_comp_c=num_comp_c)

Number of features: 134
Epoch 1/200
41/41 [==============================] - 0s 12ms/step - loss: 0.4618 - val_loss: 0.0899
Epoch 2/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0360 - val_loss: 0.0217
Epoch 3/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0215 - val_loss: 0.0202
Epoch 4/200
41/41 [==============================] - 0s 8ms/step - loss: 0.0200 - val_loss: 0.0193
Epoch 5/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0191 - val_loss: 0.0187
Epoch 6/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0185 - val_loss: 0.0184
Epoch 7/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0181 - val_loss: 0.0181
Epoch 8/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0178 - val_loss: 0.0179
Epoch 9/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0176 - val_loss: 0.0177
Epoch 10/200
41/41 [==============================] - 0s 7ms/step - loss: 0.0174 - v

In [14]:
submission.reset_index().to_csv("/kaggle/working/submission.csv", index=False)